In [ ]:
%pip install python-dotenv==1.0.1 PyYAML==6.0.2 torch==2.5.1 ultralytics==8.3.107 wandb==0.19.9

In [ ]:
from pathlib import Path
from ultralytics import YOLO
import os
import os
import torch
import wandb
import yaml

In [ ]:
# Utils

def _get_wandb_key_colab() -> str:
    try:
        from google.colab import userdata # type: ignore

        if userdata.get("WANDB_API_KEY") is not None:
            return userdata.get("WANDB_API_KEY")
        else:
            raise ValueError("No WANDB key found")
    except:
        return None

def _get_wandb_env(path: Path) -> str:
    try:
        from dotenv import dotenv_values # type: ignore

        """Get W&B API key from Colab userdata or environment variable"""

        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(f"Could not find .env file at {path}")

        print(f"Loading secrets from {path}")

        secrets = dotenv_values(path)
        print(f"Found keys: {list(secrets.keys())}")

        if "WANDB_API_KEY" not in secrets:
            raise KeyError(f"WANDB_API_KEY not found in {path}. Available keys: {list(secrets.keys())}")

        return secrets['WANDB_API_KEY']
    except:
        return None

def get_wandb_key(path: Path = "../.env") -> str:
    return _get_wandb_key_colab() if _get_wandb_key_colab() is not None else _get_wandb_env(path)

def get_device() -> str:
    try:
        return 0 if torch.cuda.is_available() else "cpu"
    except Exception as e:
        print(f"Error setting device: {e}")

In [ ]:
# Config

config_data = """
wandb:
  project: "EyeInTheSky_merged"
  group: "predict"
"""

In [ ]:
config = yaml.safe_load(config_data)

In [ ]:
# WandB init

key = get_wandb_key()
wandb.login(key=key, relogin=True)

run = wandb.init(
    save_code=True,
    **config["wandb"],
)

In [ ]:
ROOT_PATH = Path.cwd().parents[0]
MODELS_PATH = os.path.join(ROOT_PATH, "models")

In [ ]:
model = YOLO(Path(MODELS_PATH) / "dulcet-salad.pt")

In [ ]:
imgs = ["prediction-0-raw.jpg"]

In [ ]:
results = model.predict(imgs)

In [ ]:
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk
    wandb.log({"prediction": wandb.Image(result.plot())})
    print(result)

In [ ]:
wandb.finish()